In [ ]:
from common import *

In [ ]:
client = start_cluster(5, 12)
client_ip_and_port(client)

# SWE output comparison between iSNOBAL and Snow-17

## CBRFC

In [ ]:
CBRFC_CSV = pd.read_csv(
    CBRFC_DIR / 'SNOW17ALEC2_2018to2021.csv',
    parse_dates=True,
    index_col=0,
    header=0,
    names=['Lower', 'Middle', 'Upper', 'Average'],
    dtype={'Lower': np.float64, 'Middle': np.float64, 'Upper': np.float64},
)
CBRFC_CSV.Lower *= INCH_TO_MM
CBRFC_CSV.Middle *= INCH_TO_MM
CBRFC_CSV.Upper *= INCH_TO_MM

## iSNOBAL 

In [ ]:
wy_swe = xr.open_mfdataset(
    (SNOBAL_DIR / 'wy20*' / 'erw/*/snow.nc').as_posix(),
    parallel=True,
)

wy_swe.coords['mask'] = (('y', 'x'), cbrfc_zones())

In [ ]:
hlf = wy_swe.where(wy_swe.mask == ALEC2HLF).specific_mass.mean(("x", "y")).compute()
hmf = wy_swe.where(wy_swe.mask == ALEC2HMF).specific_mass.mean(("x", "y")).compute()
huf = wy_swe.where(wy_swe.mask == ALEC2HUF).specific_mass.mean(("x", "y")).compute()

In [ ]:
client.shutdown()

## Compare CBRFC to iSNOBAL

In [ ]:
x_axis_fmt = mdates.DateFormatter('%d-%b-%y')

In [ ]:
plt.figure(dpi=300, figsize=(9,3))
ax = plt.gca()

linestyle=dict(alpha=0.7, lw=1)

ax.plot([], [], ' ', label='Lower', lw=0)
hlf.plot(ax=ax, label='ISNOBAL', color='gold', **linestyle)
CBRFC_CSV.Lower.plot(ax=ax, color='orange', label='Snow-17', ls='--', **linestyle)
ax.plot([], [], ' ', label='Middle')
hmf.plot(ax=ax, label='ISNOBAL', color='teal', **linestyle)
CBRFC_CSV.Middle.plot(ax=ax, color='cadetblue', label='Snow-17', ls='--', **linestyle)
ax.plot([], [], ' ', label='Upper')
huf.plot(ax=ax, label='ISNOBAL', color='indigo', **linestyle)
CBRFC_CSV.Upper.plot(ax=ax, color='mediumorchid', label='Snow-17', ls='--', **linestyle)

ax.xaxis.set_major_formatter(x_axis_fmt)
ax.set_xlim(left=CBRFC_CSV.Upper.index.min(), right=CBRFC_CSV.Upper.index.max())
         
plt.ylabel('SWE (mm)')
plt.xlabel('Date')
         
legend = plt.legend(bbox_to_anchor=(1.18, .5), loc='center right')
         
for text in legend.get_texts():
    if text.get_text() in ['Upper', 'Middle', 'Lower']:
        text.set_fontstyle('italic')